In [ ]:
#start here this initializes the required modules

import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import ase
from ase import Atoms
from ase.io import read
from ase.build import molecule
from ase.io import gamess_us
from ase.io.gamess_us import read_gamess_us_out
from ase.visualize import view
from ase.build import molecule
from weas_widget import WeasWidget
from ase.calculators.gamess_us import GAMESSUS
from ase.io import write
import re




def clean_for_new_job(name):

    # clean up output name in current folder
    nameout= name + '.out'
    if os.path.exists(nameout):
        # If the file exists, delete it
        os.remove(nameout)
        print(f"{nameout} has been deleted.")

    #clean output in extra folders

    # Specify the directory path where the files are located
    directory_path = "/scratch/cmuhich/gamess/restart"

    # Specify the pattern of files you want to delete using wildcard characters
    pattern = name + ".*"  # Example pattern, you can modify it as needed

    # Combine the directory path and pattern to form the full pattern
    full_pattern = os.path.join(directory_path, pattern)

    # Get a list of file paths that match the full pattern
    file_paths = glob.glob(full_pattern)

    # Check if there are any files matching the pattern
    if file_paths:
        # Iterate through the list of file paths
        for file_path in file_paths:
            # Delete each file
            os.remove(file_path)
            print(f"{file_path} has been deleted.")
    else:
        print(f"No files matching the pattern {pattern} were found in {directory_path}.")

    directory_path = "/home/cmuhich/gamess_scr"

    # Combine the directory path and pattern to form the full pattern
    full_pattern = os.path.join(directory_path, pattern)

    # Get a list of file paths that match the full pattern
    file_paths = glob.glob(full_pattern)

    # Check if there are any files matching the pattern
    if file_paths:
        # Iterate through the list of file paths
        for file_path in file_paths:
            # Delete each file
            os.remove(file_path)
            print(f"{file_path} has been deleted.")
    else:
        print(f"No files matching the pattern {pattern} were found in {directory_path}.")


def get_energy(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()

    energy_pattern = re.compile(r"TOTAL ENERGY =\s+([-+]?\d*\.\d+|\d+)")
    energies = []
    for line in lines:
        match = energy_pattern.search(line)
        if match:
            energy = float(match.group(1))
            energies.append(energy)
    
    
    if energies:
        print("Final Energies:")
        for i, energy in enumerate(energies, start=1):
            print(f"Run {i}: {energy} Hartree")
    else:
        print("No energy information found in the output file.")
    
    
    return energies

def get_energy_at_a_temp(e,h_correction):
    conversion_factor = 2625.5002
    energy_kj = e * conversion_factor
    energy=energy_kj+h_correction
    return energy


In [ ]:
def get_final_structure(name):
    
    ! csplit $name /'ATOMIC BASIS SET'/
    ! mv xx00 tempxxx
    ! rm xx01
    ! csplit tempxxx /'HE ORDER OF THE PRINCIPAL'/
    ! mv xx01 tempxxx
    ! rm xx00
    ! csplit tempxxx /'CHARGE         X                   Y                   Z'/
    ! mv xx01 tempxxx
    ! rm xx00
    ! csplit tempxxx /'NTERNUCLEAR DISTANCES (ANGS.)'/
    atms_of_interest=! tail -n +2 xx00 | awk '{print $1}' #| tr '\n' " ,"

    atms_of_interest = [i for i in atms_of_interest if i]

#print(atms_of_interest)

    ! rm xx*
    ! rm tempxxx

    ! csplit $name /'LOCATED'/

    ! csplit xx01 /'NTERNUCLEAR DISTANCES (ANGS.)'/
    pos_of_interest=! tail -n +5 xx00  | awk '{print $3, $4, $5}'


    pos_of_interest = [list(map(float, item.split())) for item in pos_of_interest]
    pos_of_interest = [i for i in pos_of_interest if i]
    ! rm xx* 

    values = Atoms(symbols=atms_of_interest, positions=pos_of_interest)
    return values
#print(pos_of_interest)

In [ ]:
def print_orbital_energies(name):
#name = nameout

    print(name)
    ! tac $name > tempdathold
    ! csplit tempdathold /'EIGENVECTORS'/
    ! tac xx00 > tempdathold
    ! csplit tempdathold /'BOND ORDER THRESHOLD'/
    ! mv xx00 tempdathold
    ! csplit tempdathold /'ENERGY COMPONENTS'/
    ! mv xx00 temp_eign_info.dat
    ! mv xx01 rest_temp_dat
    print('The following are the orbital numbers and the orbital energies in Hartrees')
    ! grep -v '[a-zA-Z]' temp_eign_info.dat | grep '[0-9]' 

    ! csplit rest_temp_dat /'ATOM         MULL.POP.    CHARGE          LOW.POP.     CHARGE'/
    print('The following are the Mullikin and Lowdin charges')
    ! cat xx01

In [ ]:
#now determine how many cores we have

ncor = ! nproc | tr -d -c 0-9


slist_string = " ".join(ncor)
try:
    integer_value = int(slist_string)
    print("number of cores to use:", integer_value)
    ncor=int(integer_value/2)
    
except ValueError:
    print("Unable to determine number of cores. set this yourself in the next cell")
    ncor=[]


In [ ]:
#this will enable you to put in the number of cores avaialble
if ncor ==[]:
    print('please input the number of cores availalbe')
    ncor=input('how many cores are avaialble')
    print("number of cores to use:", ncor)
    ncor=int(ncor/2)
else:
    print('move on to the next cell')

In [ ]:
# pull up GUI for design
atoms = molecule("C6H6")
viewer = WeasWidget()
viewer.from_ase(atoms)
viewer

#atom manipulation:
# click atom or group (using the shift key)
# to move press "g" then do the motion
#to rotate press "r" then do the motion
# to duplicate press "d" 
# to delete press the delete key

In [ ]:
# store information
atoms=viewer.to_ase()

In [ ]:
#make calcuation choices
calc = GAMESSUS(contrl=dict(scftyp='rhf', dfttyp='blyp', runtyp='optimize'),
                basis=dict(gbasis='n31', ngauss=6, ndfunc=1, npfunc=1),
               statpt=dict(nstep=100))

In [ ]:
#select name and run job

name='benzene'
namein= name + '.inp'
nameout= name + '.out'


atoms.set_calculator(calc)
calc.write_input( atoms)
! mv gamess_us.inp $namein

clean_for_new_job(name)

! rungms $namein 00 $ncor >& $nameout

In [ ]:
energies=get_energy(nameout)


In [ ]:
! grep NSERCH $nameout

In [ ]:
atom =get_final_structure(nameout)
viewer.from_ase(atom)
viewer

In [ ]:
atoms=viewer.to_ase()

In [ ]:
atom = ase.io.iread(nameout)


num_positions_sets = len(atoms)
print("Number of position sets:", num_positions_sets)



for i, frame in enumerate(atom, start=1):
    print(f"Position set {i}:")
    print(frame.positions)
    print()

atom = ase.io.iread(nameout)


num_positions_sets = len(atoms)
print("Number of position sets:", num_positions_sets)



for i, frame in enumerate(atom, start=1):
    print(f"Position set {i}:")
    print(frame.positions)
    print()
    

#atom.get_positions()

In [ ]:
#vibrational spectra


#make calcuation choices
calc = GAMESSUS(contrl=dict(scftyp='rhf', dfttyp='blyp', runtyp='Hessian'),
                basis=dict(gbasis='n31', ngauss=6, ndfunc=1, npfunc=1),
                force=dict( METHOD='SEMINUM') )


#select name and run job

name='hessian'
namein= name + '.inp'
nameout= name + '.out'


atoms.set_calculator(calc)
calc.write_input( atoms)
! mv gamess_us.inp $namein

clean_for_new_job(name)

! rungms $namein 00 $ncor > $nameout


In [ ]:
#this will tell you the frequencies in cm-1 and their intensities
! grep 'FREQUENCY' $nameout

!grep 'IR INTENSITY' $nameout

In [ ]:
#this will tell you additional thermodyanmic variabls
! grep 'THERMOCHEMISTRY AT' $nameout
! grep -C 10 'E         H         G         CV        CP        S' $nameout

In [ ]:
#this will tell you the total energy at the temperature

#type in the "H" total energy from the above table in kJ/mol units
correction=273.137


energies=get_energy(nameout)
energy_at_temp=get_energy_at_a_temp(energies[0],correction)
print('the energy of ', name , ' in kJ/mol is:', energy_at_temp)

In [ ]:
print_orbital_energies(nameout)